Created on Aug 9, 2016  
@author: Xiangnan He (xiangnanhe@gmail.com)  

Modified on Dec 7, 2018  
@author: bwlee@kbfg.com  

Keras Implementation of Multi-Layer Perceptron (MLP) recommender model in:  
He Xiangnan et al. Neural Collaborative Filtering. In WWW 2017.   

#### 수정사항
* 전처리 과정 효율화
    * 기존 함수 get_train_instances의 경우 item의 negative instance 랜덤 샘플을 하나씩 수행  
      --> 각 user 별 negative instance 수집을 한 번에 전체 샘플을 선택하도록 수정
* 기존 코드는 keras v1을 사용하여 v2 에 맞춰 수정
* 기존 코드는 python2 를 사용하여 python3 에 맞게 수정

# Neural Collaborative Filtering
상품 추천에서 최근 가장 널리 쓰이는 Collaborative Filtering(CF) 방법은 user와 item 간의 선형 곱으로 둘 간의 상호작용을 표현한다.  
해당 상호작용 부분을 인공신경망으로 대체할 경우 둘 간의 연관성을 비선형모델로 분석 가능하다.  
상세한 설명은 아래에서 확인 가능하다.  
https://arxiv.org/abs/1708.05031  
https://github.com/hexiangnan/neural_collaborative_filtering  

위의 자료에서는 CF를 기존 방법인 Matrix factorization(MF) 과 MLP 방법으로 구현하고, 두 방법을 결합한 형태로 최종 결과를 구하였다.  
MF 방법 또한 인공신경망으로 구성 가능하며 ~/samples/music_recommendation 에서 적용한 방법과 동일하다.
아래 코드에서는 MLP 방법을 구현하였다.

## Data
MovieLens  
영화 평가 사이트에서 데이터를 공개하고 있으면 자료 양에 따라 몇 개의 dataset 을 제공  
이 코드는 아래 위치에서 원본 데이터를 수집 후 1차 가공한 데이터를 사용  
https://grouplens.org/datasets/movielens/1m/  

## 모델 구성
이 모델의 구성은 아래의 그림과 같다.  
일반적인 인공신경망과 비교해서 특징적인 점은 user와 item으로 부터 두 개의 별도 네트워크를 구성하고, 중간에서 합친다는 점이다.  
이를 통해 전혀 다른 두 개의 특성을 연결시키고, 두 개의 상호관계를 인공신경망을 통해 추정하는 것이다.  
<img src="MLP_ncf.PNG" width="600"/>

이 방법의 CF의 본질적인 문제점을 가지고 있는데, user 와 item 이 사전에 정해져 있어야 한다는 것이다. 신규 가입자와 신규 상품의 경우 정상적인 예측이 불가능하다.  
이를 위해서는 해당 신규 항목에 대해 어느 정도 시간 동안 데이터 축적 후 이들을 새롭게 포함하여 모델을 업데이트 하는 것이다.  


In [1]:
import numpy as np

import keras
from keras import backend as K
from keras.regularizers import l2
from keras.models import Sequential, Model
from keras.layers.core import Dense, Lambda, Activation
from keras.layers import Embedding, Input, Dense, merge, Reshape, Flatten, Dropout, Concatenate
#from keras.merge import Concatenate
from keras.constraints import maxnorm
from keras.optimizers import Adagrad, Adam, SGD, RMSprop
from keras import initializers
from evaluate import evaluate_model
from Dataset import Dataset
from time import time
import sys
import argparse
import multiprocessing as mp
from functools import partial

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


ModuleNotFoundError: No module named 'evaluate'

In [2]:
def init_normal():
    return initializers.RandomNormal(mean=0.0, stddev=0.01)

def get_model(num_users, num_items, layers = [20,10], reg_layers=[0,0]):
    assert len(layers) == len(reg_layers)
    num_layer = len(layers) #Number of layers in the MLP
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    
    user_embed = Embedding(input_dim = num_users, output_dim = int(layers[0]/2), name = 'user_embedding',
                           embeddings_initializer = initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None), input_length=1)
    item_embed = Embedding(input_dim = num_items, output_dim = int(layers[0]/2), name = 'item_embedding',
                           embeddings_initializer = initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None), input_length=1)   
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(user_embed(user_input))
    item_latent = Flatten()(item_embed(item_input))
    
    # The 0-th layer is the concatenation of embedding layers
    vector = Concatenate(1)([user_latent, item_latent])
    
    # MLP layers
    for idx in range(1, num_layer):
        #layer = Dense(layers[idx], W_regularizer= l2(reg_layers[idx]), activation='relu', name = 'layer%d' %idx)
        layer = Dense(layers[idx], activation='relu', name = 'layer%d' %idx)
        vector = layer(vector)
        
    # Final prediction layer
    prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = 'prediction')(vector)
    
    model = Model(inputs=[user_input, item_input], outputs=prediction)
    
    return model

In [3]:
def get_dict_user_item(arr_dict, n_user, n_item):
    ret = { i: [] for i in range(n_user) }
    items0 = set(range(n_item))
    rem = { i: None for i in range(n_item) }
    for key1, _ in arr_dict.items():
        ret[key1[0]].append(key1[1])
    for ii in range(n_user):
        rem[ii] = list( items0 - set(ret[ii]) )
    return ret, rem

In [13]:
def get_train_instances(train, num_negatives):
    num_user, num_item = train.shape
    user_input, item_input, labels = [], [], []
        
    # original code getting negative samples is very slow.
    # Added hash and get random negative values first
    
    # arrange train data into hash with user key
    pos_dic, neg_dic = get_dict_user_item(train, num_user, num_item)
    
    for u in range(num_user):
        if u%2000 == 0:
            print('---------u = ', u)
        pos_items = pos_dic[u]
        n_pos = len(pos_items)
        if n_pos*num_negatives < len(neg_dic[u]):
            neg_items = np.random.choice(neg_dic[u], size=(n_pos, num_negatives), replace=False)
        else:
            neg_items = []
            for i in range(n_pos):
                neg_items.append( np.random.choice(neg_dic[u], size=(num_negatives), replace=False) )

        i_pos = 0
        for i_item in pos_items:
            # positive instance
            user_input.append(u)
            item_input.append(i_item)
            labels.append(1)
            # negative instances
            for j_item in neg_items[i_pos]:
                user_input.append(u)
                item_input.append(j_item)
                labels.append(0)
            i_pos += 1
        
    return user_input, item_input, labels

In [5]:
path = 'Data/'
dataset = 'ml-1m'
layers = [64, 32, 16, 8]
reg_layers = [0, 0, 0, 0]
num_negatives = 4
learner = 'adam'
learning_rate = 0.001
batch_size = 256
epochs = 20
verbose = 1
    
topK = 10
evaluation_threads = 1 #mp.cpu_count()
model_out_file = 'Pretrain/%s_MLP_%s_%d.h5' %(dataset, layers, time())

In [6]:
# Loading data
t1 = time()
dataset = Dataset(path + dataset)
train, testRatings, testNegatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives
num_users, num_items = train.shape
print("Load data done [%.1f s]. #user=%d, #item=%d, #train=%d, #test=%d" 
        %(time()-t1, num_users, num_items, train.nnz, len(testRatings)))

Load data done [18.4 s]. #user=6040, #item=3706, #train=994169, #test=6040


In [10]:
# Build model
model = get_model(num_users, num_items, layers, reg_layers)
if learner.lower() == "adagrad": 
    model.compile(optimizer=Adagrad(lr=learning_rate), loss='binary_crossentropy')
elif learner.lower() == "rmsprop":
    model.compile(optimizer=RMSprop(lr=learning_rate), loss='binary_crossentropy')
elif learner.lower() == "adam":
    model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy')
else:
    model.compile(optimizer=SGD(lr=learning_rate), loss='binary_crossentropy')    

In [11]:
# Check Init performance
t1 = time()
(hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads)
hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
print('Init: HR = %.4f, NDCG = %.4f [%.1f]' %(hr, ndcg, time()-t1))

Init: HR = 0.1060, NDCG = 0.0493 [54.2]


In [15]:
# Train model
best_hr, best_ndcg, best_iter = hr, ndcg, -1
for epoch in range(epochs):
    t1 = time()
    # Generate training instances
    user_input, item_input, labels = get_train_instances(train, num_negatives)
    print('--------------------- fit start', epoch)
    # Training        
    hist = model.fit([np.array(user_input), np.array(item_input)], #input
                    np.array(labels), # labels 
                    batch_size=batch_size, epochs=1, verbose=0, shuffle=True)
    t2 = time()
    print('--------------------- fit end', epoch)
    # Evaluation
    if epoch %verbose == 0:
        (hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads)
        hr, ndcg, loss = np.array(hits).mean(), np.array(ndcgs).mean(), hist.history['loss'][0]
        print('Iteration %d [%.1f s]: HR = %.4f, NDCG = %.4f, loss = %.4f [%.1f s]' 
            % (epoch,  t2-t1, hr, ndcg, loss, time()-t2))
        if hr > best_hr:
            best_hr, best_ndcg, best_iter = hr, ndcg, epoch
            model.save_weights(model_out_file, overwrite=True)

print("End. Best Iteration %d:  HR = %.4f, NDCG = %.4f. " %(best_iter, best_hr, best_ndcg))
if args.out > 0:
    print("The best MLP model is saved to %s" %(model_out_file))

---------u =  0
---------u =  2000
---------u =  4000
---------u =  6000
--------------------- fit start 0
--------------------- fit end 0
Iteration 0 [582.6 s]: HR = 0.5858, NDCG = 0.3330, loss = 0.3032 [54.4 s]
---------u =  0
---------u =  2000
---------u =  4000
---------u =  6000
--------------------- fit start 1
--------------------- fit end 1
Iteration 1 [583.7 s]: HR = 0.6106, NDCG = 0.3493, loss = 0.2837 [53.7 s]
---------u =  0
---------u =  2000
---------u =  4000
---------u =  6000
--------------------- fit start 2
--------------------- fit end 2
Iteration 2 [588.0 s]: HR = 0.6280, NDCG = 0.3605, loss = 0.2731 [54.5 s]
---------u =  0
---------u =  2000
---------u =  4000
---------u =  6000
--------------------- fit start 3
--------------------- fit end 3
Iteration 3 [590.4 s]: HR = 0.6397, NDCG = 0.3679, loss = 0.2663 [54.3 s]
---------u =  0
---------u =  2000
---------u =  4000
---------u =  6000
--------------------- fit start 4
--------------------- fit end 4
Iteration

NameError: name 'args' is not defined